# sc3nb Test cases

In [ ]:
%load_ext autoreload
%autoreload 2
# The nodes in the WeakValueDict instance of the SCServer does not update to new type when using autoreload.
# So be careful with this when changing the Node classes.

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import sys, time, random, os
import numpy as np

import pythonosc.parsing.osc_types as osc_types

In [ ]:
import sc3nb as scn

In [ ]:
if 'sc' in locals() and sc is not None and isinstance(sc, scn.SC):
    sc.exit()

In [ ]:
TEST_PORT = scn.osc.osc_communication.SCSYNTH_DEFAULT_PORT+1

# start without sclang and with different port to enforce own server process creation
options = scn.ServerOptions(udp_port=TEST_PORT)
sc = scn.startup(start_sclang=False, scsynth_options=options)

In [ ]:
sc.server.osc.fails.msg_queues

In [ ]:
sc.server.max_logins

In [ ]:
sc.server.notify(True)

In [ ]:
sc.server.notify(False)

In [ ]:
sc.server.notify(True)

In [ ]:
assert sc.server.is_local, "Server is not local"

In [ ]:
sc.server.dump_osc(1)  # scsynth will print received OSC packets

In [ ]:
sc.server.sync()

## missing sclang tests

In [ ]:
assert sc._sclang is None, "_sclang was set"

In [ ]:
runtime_warning = False
try:
    sc.lang.cmd("s;")
except RuntimeWarning:
    runtime_warning = True
assert runtime_warning, "sclang access should have raised a RuntimeWarning"

## automatic (with) Bundler test

In [ ]:
try:
    del syn1
except NameError as error:
    pass 
try:
    del syn2
except NameError as error:
    pass 
try:
    del bundler
except NameError as error:
    pass 
    
# if we have syn1, syn2 in scope the automatic free messages will be included in the bundle..
# but if we dont allow bundling of synths freed by __del__ it will also affect inner bundle variable reuse 

with scn.Bundler(0.1) as bundler:
    syn1 = scn.Synth("s2", {"amp": 0.1})
    bundler.wait(0.15)
    syn2 = scn.Synth("s2", {"freq": 900, "amp": 0.1})
    bundler.wait(0.15)
    syn2.free()
    bundler.wait(0.2)
    syn1.free()

assert len(bundler.contents) == 4, f"content size does not match {bundler.contents}"
assert bundler.passed_time == 0.5, "passed time does not match"

In [ ]:
with scn.Bundler(0.1) as bundler:
    syn1 = scn.Synth("s2", {"amp": 0.1})
    bundler.wait(0.5)
    syn2 = scn.Synth("s2", {"freq": 200, "amp": 0.1})
    bundler.wait(0.5)
    syn1.free()
    bundler.wait(0.5)  # 0.5 shorter
    syn2.free()

assert len(bundler.contents) == 4, f"content size does not match {bundler.contents}"
assert bundler.passed_time == 1.5, "passed time does not match"

In [ ]:
del syn1, syn2

## Buffer tests

In [ ]:
bufdata = np.random.rand(30000, 1)
buf = scn.Buffer().load_data(bufdata)
assert np.allclose(buf.to_array(), bufdata), "Buffer data mismatch"

## synth features tests

### synth features without sclang

In [ ]:
assert sc._sclang is None, "sclang was started but shouldn't be"

In [ ]:
args = {'amp': 0.02, "num": 3}
syn1 = scn.Synth("s2", args=args)  # when a Synth param was set it should accessible without sclang
for name, value in args.items():
    assert np.allclose(syn1.__getattr__(name), value), f"{name} value mismatch"

In [ ]:
syn1.new() # assert Warning Errir from scsynth

In [ ]:
for name, value in {'amp': 0.01, "num": 1}.items():
    syn1.__setattr__(name, value)
    assert np.allclose(syn1.__getattr__(name), value), f"{name} value mismatch"

In [ ]:
attribute_error = False;
try:
    print(syn1.freq)  # when a Synth param wasnt set nor sclang provided a synth desc it should raise an AttributeError
except AttributeError:
    attribute_error = True
assert attribute_error

In [ ]:
#with self.assertWarns(UserWarning):
syn1.freq = 200  # this should seen as python attribute and not affect the sound

In [ ]:
#with self.assertWarns(UserWarning):
syn1.set("freq", 1000)  # here the python attribute should be removed and warned about it

In [ ]:
syn1.freq = 400  # this should change the synth parameter

### Synth setting in bundler

In [ ]:
syn1.set("pan", -1)
num_msgs = 1259
duration = 1.5
wait_time = duration / num_msgs
with sc.server.bundler(0.4) as bundler:
    for p in np.linspace(-1, 1, num_msgs):
        syn1.pan = p
        bundler.wait(wait_time)

In [ ]:
num_msgs = 1000
duration = 2
wait_time = duration / num_msgs
with sc.server.bundler(0.4) as bundler:
    freq = syn1.freq = 1000 
    for _ in range(num_msgs):
        freq = freq * 0.998
        syn1.freq = freq 
        bundler.wait(wait_time)

### Node Tree features without sclang

In [ ]:
import threading

def querying(server):
    while True:
        server.query_all_nodes()
        time.sleep(2)

#query_nodes_thread = threading.Thread(target=querying, args=(sc.server,))
#query_nodes_thread.start()

In [ ]:
nt = sc.server.query_all_nodes()
nt

In [ ]:
syn1_copy = scn.Synth("s1", nodeid=syn1.nodeid)
syn1_copy

In [ ]:
assert syn1 is syn1_copy, "Node Registry failed to return Node again"

In [ ]:
nt = sc.server.query_all_nodes()
nt

In [ ]:
assert len(nt.root.children) == 1, "There are other default groups"
assert nt.root.children[0] is sc.server.default_group, "The default group is not the same as the group in the NodeTree"
assert sc.server.default_group.children[0] == syn1, "Synth is not in the right group"

In [ ]:
syn1.query()

In [ ]:
default_group_info = sc.server.default_group.query()
print(default_group_info)
assert default_group_info.tail == default_group_info.head == syn1.query().nodeid, "NodeTree wrong"

In [ ]:
syn2_nodeid = 42

exception = False
try:
    syn2 = scn.Synth("s2", {"amp": 0.1}, syn2_nodeid)
except TypeError:
    exception = True
assert exception, "Creating a Synth with more than 3 positional arguments should fail"

syn2 = scn.Synth("s2", {"amp": 0.1}, nodeid=syn2_nodeid)

In [ ]:
default_group_info = sc.server.default_group.query()
print(default_group_info)
assert default_group_info.head == syn2.query().nodeid and default_group_info.tail == syn1.query().nodeid, "NodeTree wrong"

In [ ]:
sc.server.default_group.children[0].free()

In [ ]:
sc.server.default_group.query_tree()

In [ ]:
logging.basicConfig(level=logging.DEBUG)

In [ ]:
scn.Group(nodeid=1)# .query_tree()

In [ ]:
scn.Group(nodeid=1).query_tree()

In [ ]:
sc.server.free_all()

### synth features with sclang

In [ ]:
sc.start_sclang()

In [ ]:
assert sc._sclang is not None, "failed to start sclang"

In [ ]:
sclang_node_id = %scg x = Synth.new(\s2, [\freq, 100]); x.nodeID

In [ ]:
sc.server.query_all_nodes()

In [ ]:
sclang_synth = scn.Synth(nodeid=sclang_node_id, new=False)
sclang_synth

In [ ]:
sc.server.query_all_nodes()

## SynthDesc / SynthDef tests

In [ ]:
synth_desc = scn.SynthDef.get_desc("s2")
assert synth_desc is not None, "failed to get synth desc"

In [ ]:
syn = scn.Synth("s2")  # when a Synth param was set it should accessible without sclang
for name, arg in synth_desc.items():
    assert np.allclose(syn.__getattr__(name), arg.default), f"{name} value mismatch"
syn.free()

In [ ]:
if "synth_def" in locals():
    del synth_def

synth_def = scn.SynthDef(name='myklank', definition=r"""
{ |out=0, amp=0.3, freq=440|
    var klank = DynKlank.ar(`[[1,2], [1,1], [1.4,1]], Dust.ar(20), freq);
    Out.ar(out, amp*klank!2);
}""")
syn_name = synth_def.add()

In [ ]:
desc = scn.SynthDef.get_desc(syn_name)
assert desc is not None, "SynthDef is not in SynthDescLib of sclang"
assert "freq" in desc, "desc is wrong"
assert 440 == desc["freq"].default, "desc is wrong"

In [ ]:
synth = scn.Synth(syn_name)

In [ ]:
synth.free()

## Recording

In [ ]:
rec = scn.Recording()
with scn.Bundler(0.2) as bundler:
    rec.start()
    syn1 = scn.Synth("s2", {"freq": 300, "amp": 0.1})
    bundler.wait(1.0)
    syn1.free()
    rec.pause()
    bundler.wait(1.0)
    rec.resume()
    syn2 = scn.Synth("s2", {"freq": 1000, "amp": 0.1})
    bundler.wait(1.0)
    syn2.free()
    rec.stop()

In [ ]:
rec = scn.Recording(path="blip.wav")
with scn.Bundler(0.2) as bundler:
    rec.start()
    sc.server.blip()
    bundler.wait(1)
    rec.stop()

## sclang tests

### convert_to_sc tests

In [ ]:
python_list = [1,2,3,4]
assert "Array" in sc.lang.cmd("""^python_list.class""", get_output=True), "conversion failed"

In [ ]:
np_array = np.array([1,2,3,4])
assert "Array" in sc.lang.cmd("""^np_array.class""", get_output=True), "conversion failed"

### cmdg tests

In [ ]:
a = 1234
b = 23452
sc_val = sc.lang.cmdg("""^a+^b""")
assert type(sc_val) == int, "conversion failed"
assert sc_val == a+b, "value mismatch"

In [ ]:
sc_val = sc.lang.cmdg("""1234.5.squared""")
assert type(sc_val) == float, "conversion failed"
assert sc_val == 1234.5**2, "value mismatch"

In [ ]:
sc_val = sc.lang.cmdg('''"soni"++"fication"''')
assert type(sc_val) == str, "conversion failed"
assert sc_val == "soni"+"fication", "value mismatch"

In [ ]:
sc_val = sc.lang.cmdg("""(1,1.1..2)""")
assert type(sc_val) == list, "conversion failed"
assert np.allclose(sc_val, np.arange(1.0,2.0,0.1)), "value mismatch"

In [ ]:
pylist = [[1]]
sc_val = sc.lang.cmdg(f"""{pylist.__repr__()}""")
assert type(sc_val) == list, "conversion failed"
assert sc_val == pylist, "value mismatch"

In [ ]:
pylist = [[[1]]]
sc_val = sc.lang.cmdg(f"""{pylist.__repr__()}""")
assert type(sc_val) == list, "conversion failed"
assert sc_val == pylist, "value mismatch" 

In [ ]:
pylist = [[1],[1]]
sc_val = sc.lang.cmdg(f"""{pylist.__repr__()}""")
assert type(sc_val) == list, "conversion failed"
assert sc_val == pylist, "value mismatch" 

In [ ]:
pylist = [[1.0, 1.0],
 [1.100000023841858, 1.100000023841858],
 [1.2000000476837158, 1.2000000476837158],
 [1.2999999523162842, 1.2999999523162842],
 [1.399999976158142, 1.399999976158142],
 [1.5, 1.5],
 [1.600000023841858, 1.600000023841858],
 [1.7000000476837158, 1.7000000476837158],
 [1.7999999523162842, 1.7999999523162842],
 [1.899999976158142, 1.899999976158142]]
sc_val = sc.lang.cmdg(f"""{pylist.__repr__()}""")
assert type(sc_val) == list, "conversion failed"
assert sc_val == pylist, "value mismatch" 

In [ ]:
expected_controls = [['out', 'scalar', 0.0],
 ['freq', 'control', 440.0],
 ['amp', 'control', 0.10000000149011612],
 ['pan', 'control', 0.0],
 ['gate', 'control', 1.0]]
controls = sc.lang.cmdg(r"SynthDescLib.global['default'].controls.collect({ arg control; [control.name.asString, control.rate.asString, control.defaultValue]})")
assert controls == expected_controls, "controls mismatch"

## SCServer tests

### start with already used port

In [ ]:
scn.SC.default = None  # force startup to init SC again
# startup with same port to enforce double port usage
failed = False
try:
    sc = scn.startup(scsynth_options=scn.ServerOptions(udp_port=TEST_PORT)) 
except ValueError as error:
    assert "already used" in str(error), "Error seems wrong"
    failed = True
assert failed, "Server did start with already used port"

In [ ]:
sc.exit()
del sc

In [ ]:
sc = scn.startup()

In [ ]:
sc.server.blip()

## Bugs

### **BUG** bundle nesting

In [ ]:
from pythonosc.osc_bundle_builder import OscBundleBuilder
from pythonosc.osc_bundle import OscBundle

In [ ]:
bundle = scn.Bundler(0).add(0, "/s_new", ["s1", 3333, 0, 1]).add(2, "/n_free", 4444).build()
assert isinstance(bundle, OscBundle)

In [ ]:
builder = OscBundleBuilder(0)
msg_new = scn.build_message("/s_new", ["s2", 4444, 0, 1, "freq", 1000, "num", 2])
builder.add_content(msg_new)
builder.add_content(bundle)
sc.server.send(builder.build())

# Bug description: s_new nodeid 4444 will not be created, version.reply will be received from scsynth.

### **BUG** stdin write sometimes skips a command when no sleep (on windows)

In [ ]:
%sc x = Synth.new(\s2, [\freq, 100])
time.sleep(0.001)
%sc x.free

In [ ]:
%sc x = Synth.new(\s2, [\freq, 100])
%sc x.free

In [ ]:
sc.server.free_all()
%sc s.freeAll

In [ ]:
%sc x = "toast".scramble
time.sleep(0.0001)
%scg x